In [16]:
import numpy as np
from scipy import linalg
from numpy import dot

def nmf(X, latent_features, max_iter=100, error_limit=1e-6, fit_error_limit=1e-6):
    """
    Decompose X to A*Y
    """
    eps = 1e-5
    print 'Starting NMF decomposition with {} latent features and {} iterations.'.format(latent_features, max_iter)
    if not isinstance(X, np.ndarray):
        X = X.toarray()  # I am passing in a scipy sparse matrix

    # mask
    mask = np.sign(X)

    # initial matrices. A is random [0,1] and Y is A\X.
    rows, columns = X.shape
    A = np.random.rand(rows, latent_features)
    A = np.maximum(A, eps)

    Y = linalg.lstsq(A, X)[0]
    Y = np.maximum(Y, eps)

    masked_X = mask * X
    X_est_prev = dot(A, Y)
    for i in range(1, max_iter + 1):
        # ===== updates =====
        # Matlab: A=A.*(((W.*X)*Y')./((W.*(A*Y))*Y'));
        top = dot(masked_X, Y.T)
        bottom = (dot((mask * dot(A, Y)), Y.T)) + eps
        A *= top / bottom

        A = np.maximum(A, eps)
        # print 'A',  np.round(A, 2)

        # Matlab: Y=Y.*((A'*(W.*X))./(A'*(W.*(A*Y))));
        top = dot(A.T, masked_X)
        bottom = dot(A.T, mask * dot(A, Y)) + eps
        Y *= top / bottom
        Y = np.maximum(Y, eps)
        # print 'Y', np.round(Y, 2)


        # ==== evaluation ====
        if i % 5 == 0 or i == 1 or i == max_iter:
            print 'Iteration {}:'.format(i),
            X_est = dot(A, Y)
            err = mask * (X_est_prev - X_est)
            fit_residual = np.sqrt(np.sum(err ** 2))
            X_est_prev = X_est

            curRes = linalg.norm(mask * (X - X_est), ord='fro')
            print 'fit residual', np.round(fit_residual, 4),
            print 'total residual', np.round(curRes, 4)
            if curRes < error_limit or fit_residual < fit_error_limit:
                break

    return A, Y

In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame(np.random.randn(10,10))

In [25]:
A, Y = nmf(df.values, 20)

Starting NMF decomposition with 20 latent features and 100 iterations.
Iteration 1: fit residual 63.0544 total residual 67.5961
Iteration 5: fit residual 65.5972 total residual 6.7747
Iteration 10: fit residual 14.8364 total residual 16.9002
Iteration 15: fit residual 16.8647 total residual 10.4519
Iteration 20: fit residual 16.2426 total residual 15.6015
Iteration 25: fit residual 14.595 total residual 8.6697
Iteration 30: fit residual 5.2726 total residual 6.9951
Iteration 35: fit residual 3.7422 total residual 7.7985
Iteration 40: fit residual 28.2759 total residual 30.7378
Iteration 45: fit residual 29.1128 total residual 7.9033
Iteration 50: fit residual 8.9655 total residual 11.12
Iteration 55: fit residual 10.2837 total residual 8.8453
Iteration 60: fit residual 391.3904 total residual 392.5279
Iteration 65: fit residual 391.2263 total residual 6.7331
Iteration 70: fit residual 156.9913 total residual 157.5846
Iteration 75: fit residual 157.0435 total residual 7.0741
Iteration 8

In [26]:
A

array([[  6.18834605e-04,   1.63865114e-05,   1.98168045e-04,
          4.08955069e-01,   1.00000000e-05,   1.00000000e-05,
          1.62034926e-02,   1.36104813e-04,   1.00000000e-05,
          7.02407929e-03,   1.73745018e-05,   4.18770123e-03,
          1.00000000e-05,   1.30760348e-01,   1.97604013e-04,
          4.02429043e-01,   1.54849263e-04,   4.38268225e-05,
          1.42201425e-05,   3.92056258e-03],
       [  5.75424892e+00,   4.73789076e-01,   1.20764237e-01,
          2.10433599e-05,   1.00000000e-05,   2.28580749e-01,
          1.68169047e-04,   1.00000000e-05,   1.00000000e-05,
          7.71259199e-05,   5.64045342e-02,   2.28629672e-01,
          8.13219929e-02,   1.00582348e-03,   1.20965773e-01,
          7.25727518e-02,   6.65188003e-03,   7.95316875e-02,
          1.58395766e-02,   2.66858359e-02],
       [  4.73383160e-04,   1.82598580e-04,   1.76922511e+02,
          5.36667296e-03,   1.00000000e-05,   4.07012797e-05,
          2.12659057e-05,   2.39452574e-03

In [27]:
Y

array([[  1.23845862e-04,   1.99229126e-05,   3.85117638e-04,
          1.22166889e-05,   1.00000000e-05,   2.58399518e-02,
          9.46502980e-04,   1.20398256e-01,   1.78428220e-04,
          3.37379235e-05],
       [  1.05814327e+00,   1.39348743e-03,   1.00000000e-05,
          1.40435618e-04,   1.00000000e-05,   1.99227663e+00,
          8.50551176e-04,   2.06221039e-03,   1.00000000e-05,
          1.69168846e+00],
       [  4.44676909e-03,   1.00000000e-05,   1.00000000e-05,
          1.00000000e-05,   1.00000000e-05,   1.00000000e-05,
          1.00000000e-05,   9.20702062e-04,   1.00000000e-05,
          2.61541349e-04],
       [  1.39133146e-04,   1.43088299e-05,   3.98254052e-03,
          3.35669247e-04,   1.00000000e-05,   1.00000000e-05,
          1.00000000e-05,   5.13630646e-05,   1.00000000e-05,
          2.93727951e-05],
       [  2.71525851e-02,   1.15118576e+01,   6.70947325e-05,
          6.47607229e-04,   1.00000000e-05,   1.58393504e-04,
          1.00000000e-05

In [28]:
pd.DataFrame(np.dot(A, Y))

,0,1,2,3,4,5,6,7,8,9
0,0.027058,0.029626,0.002296,0.729634,3.001620,0.005506,0.429282,0.492773,0.235732,1.228710
1,0.547895,0.005062,0.311141,0.110706,3.001684,1.093819,1.573455,0.748789,0.111637,1.282215
2,0.919133,0.310579,0.007566,0.018191,3.003395,0.004655,0.352283,0.276044,0.003251,2.139601
3,0.594581,0.396370,0.008384,0.525672,3.001707,0.122775,0.538947,0.107214,0.001286,0.010151
4,0.184770,0.547087,0.000614,0.242296,3.001614,0.032123,0.394168,0.003421,0.123776,0.687210
5,0.229888,0.001824,0.004275,0.001479,3.001693,0.112409,0.264839,0.343832,0.000926,0.347725
6,0.020909,0.320374,0.009430,1.537694,3.001824,0.209893,0.327864,1.513288,0.001783,1.110493
7,0.166180,2.465077,0.008476,0.066747,3.001649,0.107940,0.576926,0.397900,0.108796,0.656644
8,0.013837,0.305425,0.861672,0.132943,3.001691,0.000338,1.021342,0.811798,0.322664,0.129164
9,0.315195,0.628875,1.017613,0.421548,3.104872,1.024542,1.913307,1.196443,0.723104,0.105439
